In [3]:
training_data = [
    ['Green', 3, 'Mango'],
    ['Yellow', 3, 'Mango'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']
]


# Add column labels to print the tree
header = ['color', 'diameter', 'label']


def unique_vals(rows, col):
    """ Find the unique values for a column in a dataset """
    return set([row[col] for row in rows])

def class_counts(rows):
    """
        Count the number of each type of example in a dataset
    """
    counts = {} # a dictionary of label --> count
    
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts
    
    
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

class Question:
    """
        A question is used to partition a dataset
        
        This class just records a column number (eg: 0 for color) and a 
        column value (ex: Green). The 'match' method is used to compare
        the feature value in an example to the feature value stored in
        the question.
    """
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
        
    def match(self, example):
        """
        Compare the feature value in an example to the feature value in
        this question
        """
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        """
        Helper function to print the question in a readable format
        """
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
        header[self.column], condition, str(self.value))
        
def partition(rows, question):
    """
    Partitions a dataset.

    For each row in the dataset, check if it matches the question.
    If so, add it to 'true rows', otherwise, add it to 'false rows'.
    """

    true_rows, false_rows = [], []

    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows
    
#######
# true_rows, false_rows = partition(training_data, Question(0, Red))
# true_rows --> all the Red rows
# false_rows --> everything else
#######
               

def gini(rows):
    """
        Calculate Gini Impurity for a list of rows
    """

    counts = class_counts(rows)
    impurity = 1

    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity
        
        
#####
# Dataset with no mixing
# no_mix = [['Mango'],
#           ['Mango']]
# this will return 0
# lot_of_mix = [['Mango'],
#               ['Mango'],
#               ['Grape'],
#               ['Grape'],
#               ['Grapefruit'],
#               ['Blueberry']]
# will return 0.8
#####
            
def info_gain(left, right, current_uncertainty):
    """
        Information Gain
        
        The uncertainty of the starting node, minus the weighted impurity
        of two child nodes
    """
    
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1-p) * gini(right)    


######
# Calculate incertainty of training data
# current_uncertainty = gini(training_data)

# How much information do we gain by partitioning on "Green"?
# true_rows, false_rows = partition(training_data, Question(0, 'Green'))
# info_gain(true_rows, false_rows, current_uncertainty)

# What about if we partitioned on 'Red' instead?
# true_rows, false_rows = partition(training_data, Question(0, 'Red'))
# info_gain(true_rows, false_rows, current_uncertainty)
    
def find_best_split(rows):
    """
    Find the best question to ask by iterating over every feature/value
    and calculating the information gain
    """
    best_gain = 0 # keep track of best info_gain
    best_question = None # Keep track of the feature/value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1 # number of columns
    """
    
    """
    for col in range(n_features):
        values = set([row[col] for row in rows]) # unique values in the column
        
        for val in values:
            question = Question(col, val)
        
            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)
        
            # Skip the split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
                
            # Calculate the IG from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            
            if gain >= best_gain: # can use > but this makes the tree look a certain way
                best_gain, best_question = gain, question
                
        return best_gain, best_question
        
        
class Leaf:
    """
        A leaf node classifies data.
        
        This holds a dictionary of class (ex: Mango) --> number of times
        it appears in the rows from the training data that reach this leaf
    """
        
    def __init__(self, rows):
        self.predictions = class_counts(rows)
            
class Decision_Node:
    """
        A decision node asks a question.
        
        This holds a reference to the question, and to the two child nodes.
        
        On the basis of this, we decide which node to add further to which branch 
    """
    
    def __init__(self, 
                question,
                true_branch,
                false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        

def build_tree(rows):
    """ Builds the tree. """
    
    # Try partitioning the dataset on each of the unique attribute
    # Calculate the IG
    # and return the question that best produces the highest gain
    gain, question = find_best_split(rows)
    
    # Base case: no further info gain
    # since we can ask no further questions, we'll return a leaf
    
    if gain ==0:
        return Leaf(rows)
    
    # If we reach here, we have found a useful feature/value to parition on
    true_rows, false_rows = partition(rows, question)
    
    # Recursively build the true branch
    true_branch = build_tree(true_rows)
    
    # Recursivley build the false branch
    false_branch = build_tree(false_rows)
    
    # Return a Question node
    # This records the best feature/value to ask at this point,
    # as well as the branches to follow
    # depending on the answer
    return Decision_Node(question, true_branch, false_branch)
        
    
def print_tree(node, spacing=""):
    
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return
    
    # Print the question at this node
    print(spacing + str(node.question))
    
    # Call this function recursively on the true branch
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + " ")
    
    # Call this function to recursivley on the false branch
    print(spacing + '--> False: ')
    print_tree(node.false_branch, spacing + " ")
    
    
def classify(row, node):
    
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions
    
    # Decide whether to follow the tru-branch or the false-branch
    # Compare the feature/value stored in the node,
    # to the example we're considering
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)
    

def print_leaf(counts):
    """ Print the predictions at a leaf """
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs
    

if __name__ == '__main__':
    my_tree = build_tree(training_data)
    
    print_tree(my_tree)
    
    # Evaluate
    testing_data = [
        ['Green', 3, 'Mango'],
        ['Yellow', 4, 'Mango'],
        ['Red', 2, 'Grape'],
        ['Red', 1, 'Grape'],
        ['Yellow', 3, 'Lemon'],
    ]
    
    for row in testing_data:
        print("Actual: %s. Predicted: %s" %
             (row[-1], print_leaf(classify(row, my_tree))))
    
        

Is color == Red?
--> True:
 Predict {'Grape': 2}
--> False: 
 Is color == Yellow?
 --> True:
  Predict {'Mango': 1, 'Lemon': 1}
 --> False: 
  Predict {'Mango': 1}
Actual: Mango. Predicted: {'Mango': '100%'}
Actual: Mango. Predicted: {'Mango': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Mango': '50%', 'Lemon': '50%'}
